# Prédiction de prix de logements

## Prérequis (Importations Bibliothèques/Fichiers) :

Importation des différentes bibliothèques (FastAi, Matplotlib, Seaborn, Numpy, Pandas, Sklearn ...) utiles à la résolution du problème :

In [ ]:
from fastai.imports import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

np.set_printoptions(linewidth=130)

Téléchargement des datasets :

In [ ]:
train_file_path = "../input/house-prices-advanced-regression-techniques/train.csv"
test_file_path = "../input/house-prices-advanced-regression-techniques/test.csv"
dataset_df = pd.read_csv(train_file_path)
testset = pd.read_csv(test_file_path)
print("La dimension du training set est de {}".format(dataset_df.shape))

## Traitement Préalable des données :

Le problème se compose donc de 1460 logements, évalués sur 81 paramètres. 
L'objectif sera d'utiliser ces colonnes pour prédire la colonne `SalePrice`.
J'ai créé 2 fonctions de prétraitement `process_num` et `process_cat` qui en plus de se séparer de la colonne `Id`, remplacent les valeurs absentes par les valeurs les plus communes/médianes et dissocient la valeur recherchée `SalePrice` des autres.

In [ ]:
dep = "SalePrice"

def process_num(df) :
    df1 = df.copy()
    df1 = df1.select_dtypes(include = ['float64', 'int64'])
    if dep in df1.columns:
        df2 = df1[[dep]]
        df1 = df1.drop(["Id",dep], axis=1)
    elif "Id" in df1.columns : 
        df2 = None
        df1 = df1.drop("Id", axis=1)
    médiane = df1.median().iloc[0]
    df1.fillna(médiane, inplace=True)
    return df1,df2

In [ ]:
def process_cat(df):
    df1 = df.copy()
    df1 = df1.select_dtypes(include = ['object'])
    df2 = df1.copy()
    #modes = df2.dropna().mode().iloc[0]
    df1.fillna("Valeur Absente", inplace=True)
    for i in df1.columns :
        df1[i] = pd.Categorical(df1[i])
    return df1

Création des nouvelles tables néttoyées :

In [ ]:
Df_trn_num = process_num(dataset_df)[0]
Df_trn_cat = process_cat(dataset_df)
Y = process_num(dataset_df)[1]
cats = Df_trn_cat.columns

Df_trn = pd.concat([Df_trn_num, Df_trn_cat], axis=1)

Df_tst_num = process_num(testset)[0]
Df_tst_cat = process_cat(testset)

Df_tst = pd.concat([Df_tst_num, Df_tst_cat], axis=1)

seed = 42
trn_df, val_df, trn_df_y, val_df_y = train_test_split(Df_trn, Y, test_size=0.2, random_state=seed)

trn_df[cats] = trn_df[cats].apply(lambda x: x.cat.codes)
val_df[cats] = val_df[cats].apply(lambda x: x.cat.codes)
Df_tst[cats] = Df_tst[cats].apply(lambda x: x.cat.codes)

## Entrainement et Evaluation du Modèle :

Création d'une fonction `rmse` (root mean square error), qui est la métrique d'évaluation de ce challenge.
NB : on utilise les logs ici comme c'est le cas dans le classement.

In [ ]:
def numper(df) :
    return np.array(df).ravel()

def rmse(df, df_y, modele):
    return np.sqrt(mean_squared_error(np.log(numper(df_y)), np.log(modele.predict(df))))

Entrainement d'un modèle type "Random Forest" sur le training set, puis évaluation sur le training et le cross validation set :

In [ ]:
rf0 = RandomForestRegressor(100, min_samples_leaf = 3)

def entrainement (df,df_y,df_val,df_val_y,rf):
    rf.fit(df, numper(df_y))
    return rmse(df,df_y,rf), rmse(df_val,df_val_y,rf)

entrainement (trn_df,trn_df_y,val_df,val_df_y,rf0)

Calcul et affichage de l'influence de chacun des paramètres :

In [ ]:
Influence_min = 0

Influence_df = pd.DataFrame(dict(cols = trn_df.columns, imp = rf0.feature_importances_));
Influence_df = Influence_df.sort_values(by='imp', ascending=True);
Influence_df = Influence_df.loc[Influence_df["imp"] >= Influence_min]
Graph_Influence = Influence_df.plot('cols', 'imp', 'barh')

In [ ]:
colonnes_influentes = Influence_df['cols'].tolist()
Influence_considérée = Influence_df['imp'].sum()
Influence_considérée

Entrainement du modèle en ne tenant compte que des paramètres ayant une influence >= `Influence_min`

In [ ]:
rf1 = RandomForestRegressor(100, min_samples_leaf = 3)
entrainement (trn_df[colonnes_influentes],trn_df_y,val_df[colonnes_influentes],val_df_y,rf1)

In [ ]:
def subm(preds, suff):
    testset[dep] = preds
    sub_df = testset[['Id',dep]]
    sub_df.to_csv(f'sub-{suff}.csv', index=False)

subm(rf0.predict(Df_tst), 'Random_Forest')